## importing all the important libraries

In [ ]:
#importing all the libraries
import pandas as pd
import Quandl
from scipy.stats import norm
import datetime
import pandas as pd
import numpy as np
import matplotlib as plt
import math

### Loading data from Quandl

In [4]:
ftse_data = Quandl.get('YAHOO/HK_4337', trim_start="2009-01-01", trim_end="2014-01-01")
ftse_data.columns = ['Open','High','Low','Close','Volume','AdjustedClose']
returns = ftse_data.AdjustedClose.pct_change()
returns.fillna(0)
mean = returns.mean()
std_dev = returns.std()

### Getting data and setting parameters

In [5]:
ftse_data['log_returns'] = ftse_data['AdjustedClose']/ftse_data['AdjustedClose'].shift()
ftse_data['log_returns'] = ftse_data.apply(lambda x: math.log(x['log_returns']), axis=1)
ftse_data = ftse_data.fillna(0)
ftse_data['mean'] = mean
ftse_data['variance'] = std_dev**2
omega = 0.000001 
alpha = 0.047
beta = 0.9466


###populating residual

In [6]:
ftse_data['residual'] = ftse_data.apply(lambda x: (x['log_returns'] - x['mean']), axis=1)

### Populating variance

In [7]:
def populate_variance(row):
    return (omega + alpha*row['residual']**2 + beta*row.shift(1)['variance'])

In [8]:
ftse_data['variance'] = ftse_data.apply(populate_variance, axis=1)

### Populating volatility

In [14]:
ftse_data['volatility'] = ftse_data.apply(lambda x: math.sqrt(x['variance']), axis=1)

In [15]:
ftse_data # Reviewing data

,Open,High,Low,Close,Volume,AdjustedClose,log_returns,mean,variance,residual,volatility
Date,,,,,,,,,,,
2009-01-01,4434.2,4434.2,4434.2,4434.2,0,4434.2,0.000000,0.000385,0.000366,-0.000385,0.019126
2009-01-02,4434.2,4561.8,4430.0,4561.8,407295500,4561.8,0.028370,0.000385,0.000403,0.027985,0.020065
2009-01-05,4561.8,4618.1,4520.8,4579.6,836676100,4579.6,0.003894,0.000385,0.000366,0.003509,0.019141
2009-01-06,4579.6,4675.7,4562.0,4638.9,1036086000,4638.9,0.012866,0.000385,0.000373,0.012480,0.019316
2009-01-07,4638.9,4638.9,4478.0,4507.5,1059568400,4507.5,-0.028735,0.000385,0.000406,-0.029120,0.020141
2009-01-08,4507.5,4514.7,4410.5,4505.4,923008000,4505.4,-0.000466,0.000385,0.000366,-0.000851,0.019127
2009-01-09,4505.4,4534.8,4431.9,4448.5,979530100,4448.5,-0.012710,0.000385,0.000374,-0.013095,0.019335
2009-01-12,4448.5,4471.3,4402.3,4426.2,785473900,4426.2,-0.005026,0.000385,0.000367,-0.005411,0.019162
2009-01-13,4426.2,4426.2,4321.3,4399.2,928429000,4399.2,-0.006119,0.000385,0.000368,-0.006504,0.019178


### Actual calculations

In [20]:
confidence_level = 0.99
quantile= 1.0 - confidence_level
volwindow = 10 # 10 day volatility
varwindow = 21 # we need 21 day rolling VaR
#21 day rolling VaR 
unnormedquantileR = pd.rolling_quantile(ftse_data['volatility'],varwindow,quantile)

returns_2=ftse_data['volatility'].shift(-1)

all_data_frame=pd.DataFrame({'returns':returns_2,
              'Rolling quantiles':unnormedquantileR
              })

portfolio_value = 1e6
concidence_level = 0.99
alpha = norm.ppf(1- concidence_level, mean, returns_2.mean())
var = portfolio_value - portfolio_value*(alpha + 1)
print "Value at Risk is $%0.2f" % var

Value at Risk is $44461.52
